In [14]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(r'C:\Users\Lenovo\Desktop\1pharmacy\docs')

In [15]:
docs = loader.load()

In [16]:
len(docs)

13

In [17]:
docs

[Document(page_content='BASIC BILL ENTRY\n\nVideo Link :\n\nhttps://drive.google.com/file/d/1-9Hq55iRbH19L1 LzvvI5oHXoGIU_i1yV/view?usp=drive_link\n\n00:00 Click on sales.\n\n00:03 Click on bill entry.\n\n00:07 Click here to add items.\n\n00:13 Select the preferred items and also there\n\n00:15 is an option for the substitute items.\n\n00:26 Choose the patient name.\n\n00:30 Choose the doctor.\n\n00:33 Click on submit.\n\n00:34 To create a bill, the bill is created.\n\n00:38 You can print and also send the bill through WhatsApp.', metadata={'source': 'C:\\Users\\Lenovo\\Desktop\\1pharmacy\\docs\\Basic Bill Entry.pdf'}),
 Document(page_content='BILL HISTORY\n\nVideo Link :\n\nhttps://drive.google.com/file/d/1QH5XFu_m7ob_wWx 8SRMn_yiRPlff1MuH/view?usp=drive_link\n\nStep-1 : Click on sales.\n\nStep-2 : Click on Bill history.\n\nStep-3 : click here to view Bill history based on the filter.\n\nStep-4 : select the appropriate filter.\n\nStep-5 : click here to view bills based on payment type

In [18]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embeddings)

In [36]:
import boto3
import botocore
from langchain.chains import RetrievalQA
from langchain.llms.bedrock import Bedrock
from langchain.prompts import PromptTemplate

config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 0}
)

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config,
)

template = """
    You are a bot for the 1P ROS System.
    Generate answers truthfully based only on the given context for the question

    Instruction:
    - Return the steps and the video link.

{context}
{question}
"""
retriever = db.as_retriever(search_type='similarity', search_kwargs={"k": 3})
llm = Bedrock(model_id="anthropic.claude-v2:1",client=bedrock_client,model_kwargs = {"temperature":1e-10,"max_tokens_to_sample": 20000})
qa_prompt = PromptTemplate(template=template, input_variables=["context","question"])
chain_type_kwargs = { "prompt": qa_prompt, "verbose": False }
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)
question="How can I create a sales bill using 1P ROS software?"
result = qa.run(question)
print(result)

 Here are the steps to create a sales bill in 1P ROS software:

1. Click on the "Sales" module
2. Select "Sales Invoice"
3. Search and select the items you want to add to the invoice
4. Fill in the required details like batch no, expiry date, quantity, etc for each item
5. Select the customer 
6. Once the customer is selected, contact details will auto-populate
7. Select the payment type 
8. Click on "Submit" to generate the sales invoice
9. The sales invoice will be created successfully

The video demonstrating the sales invoice creation process can be found here:

https://docs.google.com/document/d/1yDzzF-aSKxo5AcaTm30UAkkC_uGUo01c4BsY4hqTNB0k/edit?usp=drive_link

Follow the step-by-step process shown in the video to create a sales bill in 1P ROS. Let me know if you need any other clarification.
